In [1]:
%%configure -f
{
  "conf": {
    "spark.executor.instances": "4",
    "spark.executor.cores": "1",
    "spark.executor.memory": "2g",
    "spark.driver.memory": "4g",
    "spark.sql.shuffle.partitions": "8"
  }
}


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
664,application_1765289937462_0657,pyspark,idle,Link,Link,None,
666,application_1765289937462_0659,pyspark,idle,Link,Link,None,
667,application_1765289937462_0660,pyspark,idle,Link,Link,None,
668,application_1765289937462_0661,pyspark,idle,Link,Link,None,
669,application_1765289937462_0662,pyspark,idle,Link,Link,None,
670,application_1765289937462_0663,pyspark,idle,Link,Link,None,
671,application_1765289937462_0664,pyspark,idle,Link,Link,None,
672,application_1765289937462_0665,pyspark,idle,Link,Link,None,
673,application_1765289937462_0666,pyspark,idle,Link,Link,None,
687,application_1765289937462_0680,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col
import time

# Paths
crime_2010_path = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv"
crime_2020_path = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv"
re_codes_path   = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/RE_codes.csv"

# Schema για crime data
crime_schema = StructType([
    StructField("DR_NO",          StringType()),
    StructField("Date Rptd",      StringType()),
    StructField("DATE OCC",       StringType()),
    StructField("TIME OCC",       StringType()),
    StructField("AREA",           StringType()),
    StructField("AREA NAME",      StringType()),
    StructField("Rpt Dist No",    StringType()),
    StructField("Part 1-2",       StringType()),
    StructField("Crm Cd",         StringType()),
    StructField("Crm Cd Desc",    StringType()),
    StructField("Mocodes",        StringType()),
    StructField("Vict Age",       IntegerType()),
    StructField("Vict Sex",       StringType()),
    StructField("Vict Descent",   StringType()),
    StructField("Premis Cd",      StringType()),
    StructField("Premis Desc",    StringType()),
    StructField("Weapon Used Cd", StringType()),
    StructField("Weapon Desc",    StringType()),
    StructField("Status",         StringType()),
    StructField("Status Desc",    StringType()),
    StructField("Crm Cd 1",       StringType()),
    StructField("Crm Cd 2",       StringType()),
    StructField("Crm Cd 3",       StringType()),
    StructField("Crm Cd 4",       StringType()),
    StructField("LOCATION",       StringType()),
    StructField("Cross Street",   StringType()),
    StructField("LAT",            DoubleType()),
    StructField("LON",            DoubleType())
])

# Schema για RE_codes
re_codes_schema = StructType([
    StructField("Vict Descent",      StringType()),
    StructField("Vict Descent Full", StringType())
])

# Implementation 2: SQL API for Query 2

spark = SparkSession \
    .builder \
    .appName("SQL query 2 execution") \
    .getOrCreate()

print("====================================================================")
print("Loading crime data & RE codes as DataFrames (for SQL)...")

t0 = time.time()

# Crime DataFrames
crime_2010_df = spark.read.format("csv") \
    .options(header="true") \
    .schema(crime_schema) \
    .load(crime_2010_path)

crime_2020_df = spark.read.format("csv") \
    .options(header="true") \
    .schema(crime_schema) \
    .load(crime_2020_path)

crime_df = crime_2010_df.unionByName(crime_2020_df)

# RE_codes DataFrame
re_codes_df = spark.read.format("csv") \
    .options(header="true") \
    .schema(re_codes_schema) \
    .load(re_codes_path)

t1 = time.time()
print(f"[INFO] Data loading + union (SQL impl): {t1 - t0:.2f} sec")

# Για να αποφύγουμε column-names με κενά στο SQL, φτιάχνουμε “καθαρά” views
crimes_sql_df = crime_df.select(
    col("DATE OCC").alias("date_occ"),
    col("Vict Descent").alias("vict_descent")
)

re_codes_sql_df = re_codes_df.select(
    col("Vict Descent").alias("vict_descent"),
    col("Vict Descent Full").alias("vict_descent_full")
)

crimes_sql_df.createOrReplaceTempView("crimes")
re_codes_sql_df.createOrReplaceTempView("re_codes")

print("====================================================================")
print("SQL pipeline for Query 2 (per year, top-3 Vict Descent)...")

t2 = time.time()

query2_sql = """
WITH crime_base AS (
    SELECT
        CAST(SUBSTRING(date_occ, 1, 4) AS INT) AS year,
        vict_descent
    FROM crimes
    WHERE date_occ IS NOT NULL
      AND vict_descent IS NOT NULL
      AND vict_descent <> ''
),
grouped AS (
    SELECT
        year,
        vict_descent,
        COUNT(*) AS cnt
    FROM crime_base
    GROUP BY year, vict_descent
),
totals AS (
    SELECT
        year,
        SUM(cnt) AS total_victims
    FROM grouped
    GROUP BY year
),
with_pct AS (
    SELECT
        g.year,
        g.vict_descent,
        g.cnt,
        (g.cnt / t.total_victims) * 100.0 AS percentage
    FROM grouped g
    JOIN totals t ON g.year = t.year
),
with_labels AS (
    SELECT
        w.year,
        COALESCE(r.vict_descent_full, w.vict_descent) AS Victim_Descent,
        w.cnt       AS num_victims,
        w.percentage
    FROM with_pct w
    LEFT JOIN re_codes r
      ON w.vict_descent = r.vict_descent
),
ranked AS (
    SELECT
        year,
        Victim_Descent,
        num_victims,
        percentage,
        ROW_NUMBER() OVER (PARTITION BY year ORDER BY num_victims DESC) AS rn
    FROM with_labels
)
SELECT
    year,
    Victim_Descent,
    num_victims,
    ROUND(percentage, 2) AS percentage
FROM ranked
WHERE rn <= 3
ORDER BY year, num_victims DESC
"""

top3_per_year_sql_df = spark.sql(query2_sql)

t3 = time.time()

top3_per_year_sql_df.show(60, truncate=False)
print(f"[TIMING] SQL Query 2 pipeline: {t3 - t2:.2f} sec")


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
691,application_1765289937462_0684,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Loading crime data & RE codes as DataFrames (for SQL)...
[INFO] Data loading + union (SQL impl): 3.52 sec
SQL pipeline for Query 2 (per year, top-3 Vict Descent)...
+----+----------------------+-----------+----------+
|year|Victim_Descent        |num_victims|percentage|
+----+----------------------+-----------+----------+
|2010|Hispanic/Latin/Mexican|73558      |38.93     |
|2010|White                 |53835      |28.49     |
|2010|Black                 |33937      |17.96     |
|2011|Hispanic/Latin/Mexican|70845      |38.8      |
|2011|White                 |51219      |28.05     |
|2011|Black                 |32579      |17.84     |
|2012|Hispanic/Latin/Mexican|70338      |38.25     |
|2012|White                 |51839      |28.19     |
|2012|Black                 |33572      |18.26     |
|2013|Hispanic/Latin/Mexican|66741      |37.97     |
|2013|White                 |48453      |27.57     |
|2013|Black                 |31975      |18.19     |
|2014|Hispanic/Latin/Mexican|68763      